In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import random

def check_for_error_message(driver, error_message):
    elements = driver.find_elements(By.TAG_NAME, 'p') + driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        if error_message in element.text:
            return True
    return False

# Read phone numbers from file and remove country code
with open('test_panel_ustm.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][:2400]

'9313381793'

In [26]:
phone_numbers = ["5045099283"]
phone_numbers

['5045099283']

In [28]:

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get("https://prepaid.t-mobile.com/direct-to-account?icid=MGPR_TMO_C_CUSTSUPT_PE7I1K07DR3HZU5OA19473")

results = []
i = 0
try:
    for phone_number in phone_numbers:
        i+=1
        # Wait for the first phone number input field
        primary_phone_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tmo-input-default-17"))
        )
        primary_phone_input.clear()
        for digit in phone_number:
            primary_phone_input.send_keys(digit)
            time.sleep(random.uniform(0.1, 0.3)) # Mimic human typing

        # Wait a moment for possible error messages
        time.sleep(random.uniform(1, 2))

        # Check for the confirmation checkmark
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "i.fa.fa-check.def.active.m-r-sm"))
        )

        # Enter the phone number again 
        confirm_phone_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "tmo-input-default-18"))
        )
        confirm_phone_input.clear()
        for digit in phone_number:
            confirm_phone_input.send_keys(digit)
            time.sleep(random.uniform(0.3, 1))  # Small delay between each character

        # Click an empty space 
        empty_space = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "header"))
        )
        empty_space.click()

        # Wait for possible error messages 
        time.sleep(random.uniform(1, 2))

        # Check for all possible error messages
        if check_for_error_message(driver, "Refills and Payments are not allowed on the Tourist rate plan"):
            error_code = 3
            error_message = "Refills and Payments are not allowed on the Tourist rate plan"
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "Mobile number is invalid. Please try again."):
            error_code = 2
            error_message = "Mobile number is invalid. Please try again."
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "Mobile number is invalid."):
            error_code = 1
            error_message = "Mobile number is invalid."
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "To Refill your account using a Voucher please dial 611 from your handset or register for an online account with T-Mobile."):
            error_code = 0  # No error found
            error_message = "No error found, prepaid"
            print(i, phone_number, error_code, error_message)
        else:
            error_code = 0  # No error found
            error_message = "No error found, prepaid"
            print(i, phone_number, error_code, error_message)

        results.append((phone_number, error_code, error_message))
        
        # Refresh the page to reset state for next input
        time.sleep(random.uniform(1, 2))
        driver.refresh()

except Exception as e:
    print(f"An error occurred: {e}")
    driver.refresh()


finally:
    driver.quit()


1 5045099283 0 No error found, prepaid


In [19]:
import os

csv_file_path = 'ustm_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code', 'Error Message'])
df_results 
df_results

,Phone Number,Error Code
0,9313381793,2
1,5076614621,2
2,6127025840,2
3,7407848366,2
4,5164691945,1
5,8458735142,2
6,5165890865,2
7,8708210351,2
8,5137067701,2
9,4432865694,1


In [24]:
import os

csv_file_path = 'ustm_query_result.csv'


if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

df_final


C:\Users\ericluo\AppData\Local\Temp\ipykernel_21312\2052943241.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)


,Phone Number,Error Code
1150,9313381793,2
1151,5076614621,2
1152,6127025840,2
1153,7407848366,2
1154,5164691945,1
1155,8458735142,2
1156,5165890865,2
1157,8708210351,2
1158,5137067701,2
1159,4432865694,1


In [25]:
# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))
print(f"Results have been appended to {csv_file_path}")

Results have been appended to ustm_query_result_EL.csv
